In [2]:
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to C:\Users\Pragyan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Pragyan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...


True

In [11]:
import re

file_name = "final.txt"   # make sure this file exists in same folder

with open(file_name, 'r', encoding="utf8") as f:
    text_data = f.read().lower()
    words = re.findall(r'\w+', text_data)

vocab = set(words)

print("Total words:", len(words))
print("Unique words:", len(vocab))

Total words: 1164823
Unique words: 66684


In [12]:
def count_word_frequency(words):
    word_count = {}
    for word in words:
        word_count[word] = word_count.get(word, 0) + 1
    return word_count

word_count = count_word_frequency(words)

In [13]:
count_word_frequency("world world is so cool world")

{'w': 3, 'o': 6, 'r': 3, 'l': 4, 'd': 3, ' ': 5, 'i': 1, 's': 2, 'c': 1}

In [15]:
def calculate_probability(word_count):
    total_words = sum(word_count.values())
    return {word: count / total_words for word, count in word_count.items()}

probabilities = calculate_probability(word_count)


lemmatizer = WordNetLemmatizer()

def lemmatize_word(word):
    """Lemmatize a given word using NLTK WordNet Lemmatizer."""
    return lemmatizer.lemmatize(word)

def delete_letter(word):
    return [word[:i] + word[i+1:] for i in range(len(word))]

def swap_letters(word):
    return [word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word)-1)]

def replace_letter(word):
    letters = string.ascii_lowercase
    return [word[:i] + l + word[i+1:] for i in range(len(word)) for l in letters]

def insert_vletter(word):
    letters = string.ascii_lowercase
    return [word[:i] + l + word[i:] for i in range(len(word)+1) for l in letters]

In [ ]:
def generate_candidates(word):
    candidates = set()
    candidates.update(delete_letter(word))
    candidates.update(swap_letters(word))
    candidates.update(replace_letter(word))
    candidates.update(insert_letter(word))
    return candidates

def generate_candidates_level2(word):
    level1 = generate_candidates(word)
    level2 = set()
    for w in level1:
        level2.update(generate_candidates(w))
    return level2


In [ ]:
def get_best_correction(word, probs, vocab, max_suggestions=5):
    candidates = (
        [word] if word in vocab else list(generate_candidates(word).intersection(vocab)) or
        list(generate_candidates_level2(word).intersection(vocab))
    )
    return sorted([(w, probs.get(w, 0)) for w in candidates], key=lambda x: x[1], reverse=True)[:max_suggestions]


In [21]:
user_input = input("\n Enter a word for autocorrection: ")
suggestions = get_best_correction(user_input, probabilities, vocab, max_suggestions=5)

print("\n Top suggestions:")
for suggestion in suggestions:
    print(suggestion[0])

NameError: name 'insert_letter' is not defined

In [22]:
import nltk
import re
import string
import pickle
import os
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Download required NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

class AutoCorrectSystem:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.word_count = {}
        self.probabilities = {}
        self.vocab = set()
        self.total_words = 0
        
    def load_text_file(self, file_name):
        """Load and process text file"""
        try:
            with open(file_name, 'r', encoding="utf8") as f:
                text_data = f.read().lower()
                words = re.findall(r'\w+', text_data)
                
            self.vocab = set(words)
            self.total_words = len(words)
            self.word_count = self.count_word_frequency(words)
            self.probabilities = self.calculate_probability()
            
            print(f"Total words loaded: {self.total_words}")
            print(f"Unique words: {len(self.vocab)}")
            return True
        except FileNotFoundError:
            print(f"Error: {file_name} not found!")
            return False
    
    def count_word_frequency(self, words):
        """Count frequency of each word"""
        return dict(Counter(words))
    
    def calculate_probability(self):
        """Calculate probability of each word"""
        total = sum(self.word_count.values())
        return {word: count/total for word, count in self.word_count.items()}
    
    def lemmatize_word(self, word):
        """Lemmatize a given word"""
        return self.lemmatizer.lemmatize(word)
    
    def delete_letter(self, word):
        """Generate strings by deleting one character"""
        return [word[:i] + word[i+1:] for i in range(len(word))]
    
    def swap_letters(self, word):
        """Generate strings by swapping adjacent characters"""
        return [word[:i] + word[i+1] + word[i] + word[i+2:] 
                for i in range(len(word)-1)]
    
    def replace_letter(self, word):
        """Generate strings by replacing one character"""
        letters = string.ascii_lowercase
        return [word[:i] + l + word[i+1:] 
                for i in range(len(word)) for l in letters]
    
    def insert_letter(self, word):
        """Generate strings by inserting one character"""
        letters = string.ascii_lowercase
        return [word[:i] + l + word[i:] 
                for i in range(len(word)+1) for l in letters]
    
    def generate_candidates_level1(self, word):
        """Generate all possible level 1 candidates"""
        candidates = set()
        candidates.update(self.delete_letter(word))
        candidates.update(self.swap_letters(word))
        candidates.update(self.replace_letter(word))
        candidates.update(self.insert_letter(word))
        return candidates
    
    def generate_candidates_level2(self, word):
        """Generate level 2 candidates"""
        level1 = self.generate_candidates_level1(word)
        level2 = set()
        for w in level1:
            level2.update(self.generate_candidates_level1(w))
        return level2
    
    def get_best_correction(self, word, max_suggestions=5, use_lemmatization=True):
        """Get best corrections for a word"""
        if use_lemmatization:
            word = self.lemmatize_word(word)
        
        # Check if word is in vocabulary
        if word in self.vocab:
            return [(word, self.probabilities.get(word, 0))]
        
        # Generate level 1 candidates
        candidates = self.generate_candidates_level1(word)
        valid_candidates = candidates.intersection(self.vocab)
        
        # If no level 1 candidates, try level 2
        if not valid_candidates:
            candidates = self.generate_candidates_level2(word)
            valid_candidates = candidates.intersection(self.vocab)
        
        # Sort by probability
        suggestions = [(w, self.probabilities.get(w, 0)) 
                      for w in valid_candidates]
        suggestions.sort(key=lambda x: x[1], reverse=True)
        
        return suggestions[:max_suggestions]
    
    def add_word_to_vocab(self, word):
        """Add a new word to vocabulary"""
        word = word.lower()
        if word not in self.vocab:
            self.vocab.add(word)
            self.word_count[word] = self.word_count.get(word, 0) + 1
            self.total_words += 1
            self.probabilities = self.calculate_probability()
            return True
        return False
    
    def get_word_stats(self, word):
        """Get statistics for a word"""
        word = word.lower()
        if word in self.word_count:
            return {
                'word': word,
                'frequency': self.word_count[word],
                'probability': self.probabilities[word],
                'in_vocab': True
            }
        return {
            'word': word,
            'frequency': 0,
            'probability': 0,
            'in_vocab': False
        }
    
    def save_model(self, filename='autocorrect_model.pkl'):
        """Save the model to a pickle file"""
        model_data = {
            'word_count': self.word_count,
            'probabilities': self.probabilities,
            'vocab': self.vocab,
            'total_words': self.total_words
        }
        with open(filename, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"Model saved to {filename}")
    
    def load_model(self, filename='autocorrect_model.pkl'):
        """Load the model from a pickle file"""
        if os.path.exists(filename):
            with open(filename, 'rb') as f:
                model_data = pickle.load(f)
            self.word_count = model_data['word_count']
            self.probabilities = model_data['probabilities']
            self.vocab = model_data['vocab']
            self.total_words = model_data['total_words']
            print(f"Model loaded from {filename}")
            return True
        return False

# Create and save the model
if __name__ == "__main__":
    # Initialize the system
    autocorrect = AutoCorrectSystem()
    
    # Load your text file
    if autocorrect.load_text_file("final.txt"):
        # Save the model
        autocorrect.save_model('autocorrect_model.pkl')
        
        # Test the system
        test_words = ["worls", "programmingg", "pythoon", "computr"]
        print("\nTesting Autocorrect System:")
        for word in test_words:
            suggestions = autocorrect.get_best_correction(word)
            print(f"\nWord: {word}")
            print(f"Suggestions: {[s[0] for s in suggestions]}")
    else:
        print("Please ensure 'final.txt' exists in the current directory")

[nltk_data] Downloading package wordnet to C:\Users\Pragyan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Pragyan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Pragyan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to C:\Users\Pragyan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Total words loaded: 1164823
Unique words: 66684
Model saved to autocorrect_model.pkl

Testing Autocorrect System:

Word: worls
Suggestions: ['words', 'world', 'works', 'worlds', 'worms']

Word: programmingg
Suggestions: ['programming']

Word: pythoon
Suggestions: []

Word: computr
Suggestions: ['computer', 'compute']
